In [1]:
import numpy as np
from pyscf import gto, scf, df

In [67]:
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='STO-3G', charge = 1)
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='pc-3', charge = 1)
#mol = gto.M(atom='C 0 0 0; O 0 0 1.128', basis='STO-3G', charge = 0)
mol = gto.M(atom='H 0 0 0; H 0 0 0.740848149', basis='STO-3G', charge = 0)
#mol = gto.M(atom="O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116", basis = "sto-3g")

In [68]:
#calculate nuclear repulsion energy
tot_nuc_energy=0
for i in range(0,len(mol.atom_coords())-1):
    for j in range(0,len(mol.atom_coords())):
        if j > i:
            print("i",i,"-- j",j)
            x=mol.atom_coord(i)-mol.atom_coord(j)
           # print("x",x)
            norm=np.linalg.norm(x)
           # print("norm",norm)
            nuc_energy=mol.atom_charge(i)*mol.atom_charge(j)/norm
            print("nuc_energy",nuc_energy)     
            tot_nuc_energy+=nuc_energy
print("tot_nuc_energy",tot_nuc_energy)

i 0 -- j 1
nuc_energy 1.05835442184
tot_nuc_energy 1.05835442184


In [69]:
#get integrals and calculate core Hamiltonian

overlap_integrals = mol.intor('int1e_ovlp_sph')
#print("overlap_integrals")
#print(overlap_integrals)
kinetic_energy_integrals = mol.intor('int1e_kin_sph')
#print("kinetic_energy_integrals")
#print(kinetic_energy_integrals)
nuclear_attraction_integrals = mol.intor('int1e_nuc_sph')
#print("nuclear_attraction_integrals")
#print(nuclear_attraction_integrals)
two_electron_integrals = mol.intor('int2e_sph')
#print("two_electron_integrals")
#print(two_electron_integrals)
H_core = kinetic_energy_integrals + nuclear_attraction_integrals

In [70]:
#diagonalize overlap matrix
overlap_diag_procedure = np.linalg.eigh(overlap_integrals)
#get U and S^-0.5
U = overlap_diag_procedure[1]
s_inverse_sqrt = np.diag(1/overlap_diag_procedure[0])**0.5
#get transformation matrix X
X = np.matmul(U,np.matmul(s_inverse_sqrt,np.matrix.transpose(U)))

In [71]:
#set guess for P matrix, default core hamiltonian
P = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

In [72]:
#start iterations
counter = 0
maxiter = 100
energy_difference=10000
energy_last_iter=10000
energy=0

for counter in range(0,maxiter):
    G = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    G[i,j]+=P[k,l]*(two_electron_integrals[i,j,k,l]-0.5*two_electron_integrals[i,l,k,j])

    F = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    F = H_core + G
#    print(F)
    
    if counter >= 1:
        energy = 0
        for i in range(0,overlap_integrals.shape[0]):
            for j in range(0,overlap_integrals.shape[0]):
                energy += 0.5*(P[i,j]*(H_core[i,j]+F[i,j]))
        energy_difference = abs(energy-energy_last_iter)
        print('iteration {:<9}'.format(counter),"energy = {:<25}".format(energy), "difference = {:<23}".format(energy_difference))
        energy_last_iter = energy
    if energy_difference < 10**-8:
        print("energy converged")
        SCF_final_energy = energy + tot_nuc_energy
        print("final SCF energy including nuclear repulsion:",SCF_final_energy)
        break
    counter+=1    

    F_prime = np.matmul(np.matrix.transpose(X),np.matmul(F,X))
    F_diag_procedure = np.linalg.eigh(F_prime)
    
    C = np.matmul(X,F_diag_procedure[1])
    
    P_new = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    
    for i in range(0,overlap_integrals.shape[0]):
        for j in range(0,overlap_integrals.shape[1]):    
            for a in range(0,mol.nelectron//2):
                P_new[i,j]+=2*C[i,a]*C[j,a]

    P = P_new

iteration 1         energy = -3.8697353864597033       difference = 10003.869735386459     
iteration 2         energy = -3.9089613943170645       difference = 0.03922600785736119    
iteration 3         energy = -3.91118621602481         difference = 0.0022248217077454058  
iteration 4         energy = -3.91127236370136         difference = 8.614767655013011e-05  
iteration 5         energy = -3.9112753920347405       difference = 3.0283333805058987e-06 
iteration 6         energy = -3.911275496453328        difference = 1.0441858755427802e-07 
iteration 7         energy = -3.9112755000406887       difference = 3.587360630774583e-09  
energy converged
final SCF energy including nuclear repulsion: -2.852921078200689


In [73]:
F_diag_procedure = np.linalg.eigh(F_prime)
F_diag_procedure

(array([-1.48073791, -0.30053173]), array([[-0.93042489,  0.36648263],
        [-0.36648263, -0.93042489]]))

In [74]:
for i in range(0,2):
    print(C[i,0])

-0.9112291659538847
-0.18803143049629598


In [75]:
C

array([[-0.91122917,  0.59257588],
       [-0.18803143, -1.07057412]])

In [76]:
def integraltransformation(I, C):
    # N^5 scaling
    K = np.einsum('pi,ijkl->pjkl', C, I)
    K = np.einsum('qj,pjkl->pqkl', C, K)
    K = np.einsum('rk,pqkl->pqrl', C, K)
    K = np.einsum('sl,pqrl->pqrs', C, K)
    return K

In [77]:
C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))

for i in range(0,C_ao_mo.shape[0]):
    for p in range(0,C_ao_mo.shape[1]):
        C_ao_mo[i,p]=C[i,p]
        
mo_integrals = integraltransformation(two_electron_integrals,C_ao_mo)
mo_integrals

array([[[[ 0.51380563,  0.04473225],
         [ 0.04473225,  0.59369196]],

        [[ 0.04473225,  0.14073228],
         [ 0.14073228, -0.22164568]]],


       [[[ 0.04473225,  0.14073228],
         [ 0.14073228, -0.22164568]],

        [[ 0.59369196, -0.22164568],
         [-0.22164568,  1.31145752]]]])

In [78]:
B = np.zeros((mol.nelectron//2,overlap_integrals.shape[0]-mol.nelectron//2,mol.nelectron//2,overlap_integrals.shape[0]-mol.nelectron//2))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,overlap_integrals.shape[0]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,overlap_integrals.shape[0]):
                print(a,r,b,s)
                B[a,(r-mol.nelectron//2),b,(s-mol.nelectron//2)]+=mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a]

0 1 0 1


In [79]:
B

array([[[[0.]]]])

In [80]:
A = np.zeros((mol.nelectron//2,overlap_integrals.shape[0]-mol.nelectron//2,mol.nelectron//2,overlap_integrals.shape[0]-mol.nelectron//2))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,overlap_integrals.shape[0]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,overlap_integrals.shape[0]):
                #print(a,r,b,s)
                A[a,(r-mol.nelectron//2),b,(s-mol.nelectron//2)]+=mo_integrals[r,b,a,s]-mo_integrals[r,b,s,a]
                if a==b and r==s:
                    A[a,(r-mol.nelectron//2),b,(s-mol.nelectron//2)]+= (F_diag_procedure[0][r] - F_diag_procedure[0][a])


In [81]:
A

array([[[[1.18020618]]]])

In [82]:
mat = np.matmul((A+B),(A-B)) 

In [83]:
mat

array([[[[1.39288662]]]])

asdf

In [20]:
ntegrals_ijks = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_ijrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_iqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_pqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))


for i in range(0,C_ao_mo.shape[0]):
    for p in range(0,C_ao_mo.shape[1]):
        C_ao_mo[i,p]=C[i,p]


for s in range(0,integrals_ijks.shape[0]):
    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    integrals_ijks[i,j,k,s]+=C_ao_mo[l,s]*two_electron_integrals[i,j,k,l]
    
for s in range(0,integrals_ijrs.shape[0]):
    for r in range(0,integrals_ijrs.shape[1]):
        for i in range(0,two_electron_integrals.shape[0]):
            for j in range(0,two_electron_integrals.shape[1]):
                for k in range(0,two_electron_integrals.shape[2]):
                    integrals_ijrs[i,j,r,s]+=C_ao_mo[k,r]*integrals_ijks[i,j,k,s]

for s in range(0,integrals_iqrs.shape[0]):
    for r in range(0,integrals_iqrs.shape[1]):
        for q in range(0,integrals_iqrs.shape[2]):
            for i in range(0,two_electron_integrals.shape[0]):
                for j in range(0,two_electron_integrals.shape[1]):
                    integrals_iqrs[i,q,r,s]+=C_ao_mo[j,q]*integrals_ijrs[i,j,r,s]

for s in range(0,integrals_pqrs.shape[0]):
    for r in range(0,integrals_pqrs.shape[1]):
        for q in range(0,integrals_pqrs.shape[2]):
            for p in range(0,integrals_pqrs.shape[3]):
                for i in range(0,two_electron_integrals.shape[0]):
                    integrals_pqrs[p,q,r,s]+=C_ao_mo[i,p]*integrals_iqrs[i,q,r,s]
                    
    
#for s in range(0,integrals_pqrs.shape[0]):
 #   for r in range(0,integrals_pqrs.shape[1]):
  #      for q in range(0,integrals_pqrs.shape[2]):
   #         for p in range(0,integrals_pqrs.shape[3]):             
print(integrals_pqrs)



NameError: name 'integrals_ijks' is not defined

In [ ]:
B = np.zeros((mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0]))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,B.shape[1]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,B.shape[3]):
                B[a,r,b,s]+=mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a]

In [ ]:
A = np.zeros((mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0]))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,B.shape[1]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,B.shape[3]):
                A[a,r,b,s]+=mo_integrals[r,b,a,s]-mo_integrals[r,b,s,a]
                if a==b and r==s:
                    A[a,r,b,s]+= (F_diag_procedure[0][r] - F_diag_procedure[0][a])

#for i in range(0,A.shape[0]):
#    A[i,i,i,i]=A[i,i,i,i]+